*The code snippet assumes Anaconda 5.2.0 version of Python virtual environment*

<div class="alert alert-info">
    <h4>Acknowledgement</h4>
    <p>I would like to acknowledge <a href="http://www.michaelpyrcz.com/">Micahel Pyrcz</a>, Associate Professor at the University of Texas at Austin in the Petroleum and Geosystems Engineering, for developing course materials that helped me write this article.</p>
    <p>Check out his <a href="https://www.youtube.com/watch?v=jVRLGOsnYuw">Youtube Lecture on Variogram</a>, and <a href="https://github.com/GeostatsGuy/ExcelNumericalDemos/blob/master/Variogram%20Calc_Model_Demo_v2.0.xlsx">Variogram Excel numerical demo</a> on his Github repo to help yourself better understand the statistical theories and concepts.</p>
</div>

Let's say that you are a spatial data analyst of a gold mining company, and want to know the distribution of gold percentage over 100m x 100m mining area. To understand the characteritics of the rock formations, you take 100 random rock samples from the mining area, but obviously these 100 data points are not enough to estimate gold percentage over every single spatial locations in the area. So you analyze the available data (100 rock samples from random locations) and simulate full 2D-surface plot for gold percentage over the mining area.

 
<div class="row give-margin">
    <div class="col"><img src="jupyter_images/gold_transform.png"></div>
</div>

This 2D surface simulation from sparse spatial data is a sequential process that involved a series of geostatistical techniques. 

Steps:

1. Plot experimental variogram
2. Fit variogram model
3. Apply kriging
4. Apply simulation on top of Kriging
5. Run simulation multiple times and perform additioanl data analyses as needed

In this post, the concepts, theory, and methodology of plotting a **variogram** will be covered. 

## Experimental Variogram: Theory

> **Variogram** is a measure of dissimilarity over a distance. It shows how two data points are correlated from a spatial perspective, and provides useful insights when trying to estimate the value of an unknown location using collected sample data from other locations.

[Tobler's first law of geography](https://en.wikipedia.org/wiki/Tobler%27s_first_law_of_geography) states that "everything is related to everything else, but near things are more related than distant things." Variogram shows the correlation between two spatial data points over distances. For example, terrains 1 km apart from each other are more likely to be similar than terrains 100 km apart from each other. Oil wells 500 ft apart from each other are more likely to show similar reservoir characteristics than oil wells 5000 ft apart from each other.  

Variogram is a function of variance over distance. It has the following equation and plot:

<div class="row">
    <div class="col-12">
        $$\gamma(h) = \frac{1}{2N(h)}\sum_{\alpha =1}^{N(h)}\left ( z(u_{\alpha })-z(u_{\alpha} + h) \right)^2$$ 
    </div>
</div>
<div class="row give-margin-inline-big-plot">
    <div class="col-12"><img src="jupyter_images/basic_variogram.png"></div>
</div>

<p><u>Variables Explained</u></p>

$\gamma(h)$ = a measure of dissimilarity vs distance. It is a spatial variance between two data points separated by the distance, $h$.

$N(h)$ = number of all data point pairs separated by the distance, $h$.

$h$ = lag distance. Separation between two data points.

$u_{\alpha }$ = data point on 2D or 3D space at the location, $\alpha$.

$u_{a} + h$ = data point separated from $u_{\alpha }$ by the distance, $h$.

$z(u_{\alpha })$ = numerical value of data point, $u_{\alpha }$

$z(u_{\alpha} + h)$ = numerical value of data point, $u_{\alpha} + h$

$\sigma^2$ = sill. Variance at lag distance, $h$, in which spatial data pairs lose correlation.

<hr>

**Observation 1:** $z(u_{\alpha })$ - $z(u_{\alpha} + h)$

<div class="row give-margin-inline-plot">
    <div class="col-12"><img src="jupyter_images/grid_1.png" style="border: 1px solid;"></div>
</div>

There are two data points on the image: $z(u_{\alpha })$ and $z(u_{\alpha } + h)$. These two points are separated by the lag distance, $h$. The equation for variogram observes the difference between these two data points:

$$z(u_{\alpha })-z(u_{\alpha} + h)$$  

**Observation 2:** $N(h)$

<div class="row give-margin-inline-plot">
    <div class="col-12"><img src="jupyter_images/grid_2.png" style="border: 1px solid;"></div>
</div>

$N(h)$ accounts for <u>all</u> data point pairs that are separated by lag distance $h$. Although only horizontal separation is shown in the image, separation between two data points can be horizontal, vertical, or diagonal. Variogram will calculate the difference between all pairs of data points, $z(u_{\alpha })-z(u_{\alpha} + h)$, that are separated lag distance, $h$.

$$\sum_{\alpha =1}^{N(h)}\left ( z(u_{\alpha })-z(u_{\alpha} + h) \right)^2$$

**Observation 3:** $\gamma (h)$

$\gamma (h)$ denotes for variability of spatial data points at a lag distance, $h$. Recall that variogram accounts for <u>all</u> pairs separated by distance, $h$. It may seem very simple, but one little dot on a variogram plot is actually obtained after iterating for all pairs separated by $h$. 

$\underline{ h = 1m }$
<div class="row">
    <div class="col-12"><img src="jupyter_images/grid_3.png"></div>
</div>

$\underline{ h = 2m }$
<div class="row">
    <div class="col-12"><img src="jupyter_images/grid_4.png"></div>
</div>

$\underline{ h = 3m }$
<div class="row">
    <div class="col-12"><img src="jupyter_images/grid_5.png"></div>
</div>

Observe how there were less data pairs connected by red lines for $h = 3m$. As the $h$ increases, there will be fewer number of pairs that are separated by $h$ due to spatial limitation.

**Observation 4:** Sill ($\sigma^2$)

Sill ($\sigma^2$) is the variance in which spatial data pairs lose correlation. As the distance between two data points increases, it will be less likely that those two data points are related to one another. You may assume that the oil wells separated by 100 ft exibit similar geologic characteristics, but you can't assume the same for a well in Texas and a well in California. Variogram works the similar way.

<div class="alert alert-info">
    <h4>Notes</h4>
    <p>Spatial variance may never reach the sill if there is a trend. Ex: Area trend between well variability</p>
</div>

**Observation 5:** Range ($a$)

Range is a distance in which the spatial variability reaches the sill ($\sigma^2$). Let's say that you are an exploration engineer for drilling a new oil well. You have drilled wells A, B, C, D that are each 100ft, 200ft, 300ft, and 400ft apart from the zone you want to drill a next new well, and want to know if you can use the data from the previously drilled wells. The geostatisticians in your team report that the geologic formation in the region has a <u>range</u> of 350 ft. This means that the rocks in the region lose geologic correlation with one another if they are more than 350 ft apart — you can't use data from well D because it is 400 ft apart.

**Observation 6:** Nugget Effect ($c_{0}$)

The nugget effect refers to the nonzero intercept of the variogram and is an overall estimate of error caused by measurement inaccuracy and environmental variability occurring at fine enough scales to be unresolved by the sampling interval. At distance $h = 0$, we would expect the spatial variance between pairs will be zero, but some variables seem to change in an abrupt manner in very short distance. The nugget effect is like the random noise. It's just the small scale variability that you can't estimate with your large scale variability model.

However, if there is no expectation of high degree of discontinuity at distances shorter than the minimum data spacing, experts tend to ignore nugget effect ub geologic engineering.

<div class="alert alert-info">
    <h4>Summary</h4>
    <p>In variogram, low variance ($\gamma$) represents stronger correlation among data pairs. Spatial data pairs lose correlation with one another when variance ($\gamma$) at lag distance ($h$) reaches the sill ($\sigma^2$). If there are variance poitns that exceeds the sill, it indicates the presence of trend, and needs to be detrended before variogram modeling. More information about trend will be discussed later.</p>
</div>

## Experimental Variogram: Search Template Parameters

Calculating variogram is challenging because real-life data are not as clean as the 2-D grid sample images shown above. Real-life data are often sparse data that are irregularly spaced. The sparse data you will get in real-life will have very few, or even no data point pairs that are EXACTLY $h$ distance apart from each other. Furthermore, the data points will not always be orthogonal or pararell to each other — there will be diagonalities. 

<div class="row give-margin-inline-big-plot">
    <div class="col-md-6 col-sm-12">
        <div class="col-12"><img src="jupyter_images/spacing_regurlar.png"></div>
        <div class="col-12"><p class="image-description">regular Spacing</p></div>
    </div>
    <div class="col-md-6 col-sm-12">
        <div class="col-12"><img src="jupyter_images/spacing_irregurlar.png"></div>
        <div class="col-12"><p class="image-description">irregular Spacing</p></div>
    </div>
</div>

So how do we get pairs separated by lag distance, $h$? We need to consider distance, tolerance, azimuth direction, azimuth tolerance, dip direction, dip tolerance, bandwith in horizontal plane, and bandwidth in vertical plane — through a set of guidelines called **Variogram Search Template**.


<div class="row give-margin-inline-big-plot">
    <div class="col-md-6 col-sm-12">
        <div class="col-12"><img src="jupyter_images/search_template_1.png"></div>
    </div>
    <div class="col-md-6 col-sm-12">
        <div class="col-12"><img src="jupyter_images/search_template_2.png"></div>
    </div>
    <div class="col-12"><p class="image-description">variogram Search Template</p></div>
</div>

**Choice of Azimuth (Directionality)**

<div class="row give-margin-inline-plot">
    <div class="col-12"><img src="jupyter_images/variogram_step_1.png"></div>
    <div class="col-12"><p class="image-description">ellipsoidal growth in orthogonal directions, 3D</p></div>
</div>

Natural process does not lead to omnidirectionality. Typically there is a direction of major continuity, such as statigraphic surface, and a direction minimum continuity such as perpendicular layers. The choice of azimuth is carefully decided after combining knowledge of geologists and geologic understanding of the region from previously sampled data. 

There are three orthogonal directions — horizontal major, horizontal minor, and vertical. The "vertical" directional is assumed to be the direction orthogonal to the horizontal plane; the vertical direction doesn't have to correspond to the Z-axis. The three mutually orthogonal directions grow in ellipsoidal shape, with the horizontal major axis constituting the longer radius the ellipse. 

Azimuth decides the direction of horizontal major axis. In typical geometry, an angle ($\theta$) is measured from the positive X-axis, but in variogram, azimuth is measured from the positive Y-axis.

<div class="row give-margin-inline-plot">
    <div class="col-12"><img src="jupyter_images/variogram_azimuth_axis.png"></div>
</div>

<div class="alert alert-info">
    <h4>Notes</h4>
    <p>According to an article written by <a href="http://geostatisticslessons.com/pdfs/variogramparameters.pdf">Jared Deutsch</a>, in the absence of strong geologic evidence, a neutral isotropic model can be constructed to assist in determining a principle direction (in later code implementation, isotropy is established by setting <code>azi_tol=90</code>. You are allowing for $\pm$90 degrees of azimuth tolerance). Using ordinary kriging, a neutral model would be constructed with an isotropic, long range, high nugget effect variogram. You will check if you can observe any directionality in this neutral model, and combine it with other geologic knowledge to assist in determinng the principle direction.
    </p>
</div>

**Choice of Azimuth Tolerance**

Azimuth tolerance should be chosen such that maximum number of pairs are found in the search template and exclude unreasonable associations. A common choice of azimuth tolerance is 22.5&deg;. However, it can be changed to make a decision about whether to increase precision, or stability. Reducing angle tolerance will give precise variogram (*figure 1*), and increasing angle tolerance will give stable variogram (*figure 2*).

<div class="row">
    <div class="col-md-6 col-sm-12">
        <div class="col-12"><img src="jupyter_images/variogram_azi_tol_low.png"></div>
        <div class="col-12"><p class="image-description">figure 1: smaller azimuth tol -> precise</p></div>
    </div>
    <div class="col-md-6 col-sm-12">
        <div class="col-12"><img src="jupyter_images/variogram_azi_tol_high.png"></div>
        <div class="col-12"><p class="image-description">figure 2: bigger azimuth tol -> stable</p></div>
    </div>
</div>

**Choice of Lag Distance**

A variogram should span less than the maximum length of the field. For example, if the field A has a dimension of 100km x 100km, the maximum lag distance ($h$) of the calculated variogram should be less than 50 km. Calculating longer lags results in pairing samples from the edges of the field with each other. Due to spatial limitation of the field (recall that field A is only 100km long), there will be fewer pairs that are 80, 90, 100km apart than pairs that are 5, 10, 20km apart from each other. These variogram points will be much less informed than shorter distance pairs, with fewer pairs supporting the calculation. 

Observe how the *figure 4* shows misleading information about the region. The figure tells us that the correlation becomes stronger for pairs separated by $h > 50$ as their distance increases. This is inconsistent with natural process — we expect the spatial correlation among pairs to decrease as they are further apart from each other. 

<div class="row">
    <div class="col-md-4 col-sm-12 no-padding">
        <div class="col-12 no-padding mobile_responsive_plot"><img src="jupyter_images/variogram_lag_sample_data.png"></div>
        <div class="col-12"><p class="image-description">figure 3: sample Data</p></div>
    </div>
    <div class="col-md-4 col-sm-12 no-padding">
        <div class="col-12 no-padding mobile_responsive_plot"><img src="jupyter_images/variogram_lag_domain_length_full.png"></div>
        <div class="col-12"><p class="image-description">figure 4: total lag = field length</p></div>
    </div>
    <div class="col-md-4 col-sm-12 no-padding">
        <div class="col-12 no-padding mobile_responsive_plot"><img src="jupyter_images/variogram_lag_domain_length_half.png"></div>
        <div class="col-12"><p class="image-description">figure 5: total lag = field length / 2</p></div>
    </div>
</div>

Increasing the number of lag distance will result in fewer lag bins, which means that there will be fewer points on the calculated variogram (*figure 6*). There will be a trade-off between stability and precision, and an engineer must carefully choose the parameters that is the best representative of the geology of the region.

<div class="row">
    <div class="col-md-6 col-sm-12">
        <div class="col-12"><img src="jupyter_images/variogram_lag_bins_few.png"></div>
        <div class="col-12"><p class="image-description">figure 6: stable, but imprecise</p></div>
    </div>
    <div class="col-md-6 col-sm-12">
        <div class="col-12"><img src="jupyter_images/variogram_lag_bins_many.png"></div>
        <div class="col-12"><p class="image-description">figure 7: precise, but unstable</p></div>
    </div>
</div>

**Choice of Lag Tolerance**

Lag tolerance is usually half of the lag distance (*figure 8*). Choosing lag tolerance smaller than 1/2 lag distance will result in missing out data pairs that are not within the lag tolerance (*figure 9*). Choosing lag tolerance bigger than 1/2 lag distance will result in overlapping of data pairs (*figure 10*). 

<div class="row">
    <div class="col-md-4 col-sm-12">
        <div class="col-12 no-padding mobile_responsive_plot"><img src="jupyter_images/variogram_lag_tol_ideal.png"></div>
        <div class="col-12"><p class="image-description">figure 8: lag tol. = 1/2 lag dist.</p></div>
    </div>
    <div class="col-md-4 col-sm-12">
        <div class="col-12 no-padding mobile_responsive_plot"><img src="jupyter_images/variogram_lag_tol_small.png"></div>
        <div class="col-12"><p class="image-description">figure 9: lag tol. &lt; 1/2 lag dist.</p></div>
    </div>
    <div class="col-md-4 col-sm-12">
        <div class="col-12 no-padding mobile_responsive_plot"><img src="jupyter_images/variogram_lag_tol_big.png"></div>
        <div class="col-12"><p class="image-description">figure 10: lag tol. &gt; 1/2 lag dist.</p></div>
    </div>
</div>

Although the lag tolerance is usually half of the lag distance, in cases of erratic variograms, we may choose to overlap calculations by setting lag tolerance that is greater than half of the lag distance to increase stability in a variogram. Overlapping calculations has an effect of smoothing out and reducing noise in the calculated variogram (*figure 12*). Smoothing out the calculated variogram maybe helpful in fitting a variogram model later. However, be careful not to smooth out too much, as it will result in imprecise variogram model that is not representative of the regional geology. A decision between stability and precision must be made.

<div class="row">
    <div class="col-md-6 col-sm-12">
        <div class="col-12"><img src="jupyter_images/variogram_lag_tol_noisy.png"></div>
        <div class="col-12"><p class="image-description">figure 11: noisy</p></div>
    </div>
    <div class="col-md-6 col-sm-12">
        <div class="col-12"><img src="jupyter_images/variogram_lag_tol_smooth.png"></div>
        <div class="col-12"><p class="image-description">figure 12: smooth</p></div>
    </div>
</div>

**Choice of Bandwidth**

Just like azimuth tolerance, bandwidth should be chosen such that maximum number of pairs are found in the search template and exclude unreasonable associations. According to [Jared Deutsch](http://www.geostatisticslessons.com/pdfs/variogramparameters.pdf), bandwidths are infrequently used as they seldom improve the stability of the calculaed variogram. Instead, a carefully chosen set of angle tolerances (azimuth and dip) are applied. 

## Experimental Variogram: Python Implementation

